In [1]:
import time
from sklearn import preprocessing
from scipy.signal import butter, lfilter
import scipy.io as sio
import numpy as np
import os
import math
import matplotlib.pyplot as plt
import pandas as pd
import scipy

In [2]:
def lyapunov_wolf(data, N, m, tau, P):
    """
    %  该函数用来计算时间序列的最大Lyapunov 指数--Wolf 方法
    %  m: 嵌入维数
    %  tau:时间延迟
    %  data:时间序列
    %  N:时间序列长度
    %  P:时间序列的平均周期,选择演化相点距当前点的位置差，即若当前相点为I，则演化相点只能在|I－J|>P的相点中搜寻
    %  lambda_1:返回最大lyapunov指数值
    """
    min_point = 1  # 要求最少搜索到的点数
    MAX_CISHU = 5  # 最大增加搜索范围次数
    max_d = 0  # 最大相点距离
    min_d = 1.0e+100  # 最小相点距离
    avg_dd = 0
    Y = reconstitution(data, m, tau)  # 相空间重构
    M = N - (m - 1) * tau  # 重构相空间中相点的个数
    for i in range(M - 1):
        for j in range(i + 1, M):
            d = 0
            for k in range(m):
                d = d + (Y[k][i] - Y[k][j]) * (Y[k][i] - Y[k][j])
            d = math.sqrt(d)
            if max_d < d:
                max_d = d
            if min_d > d:
                min_d = d
            avg_dd = avg_dd + d
    avg_d = 2 * avg_dd / (M * (M - 1))  # 平均相点距离

    dlt_eps = (avg_d - min_d) * 0.02  # 若在min_eps～max_eps中找不到演化相点时，对max_eps的放宽幅度
    min_eps = min_d + dlt_eps / 2  # 演化相点与当前相点距离的最小限
    max_eps = min_d + 2 * dlt_eps  # 演化相点与当前相点距离的最大限

    # 从P+1～M-1个相点中找与第一个相点最近的相点位置(Loc_DK)及其最短距离DK
    DK = 1.0e+100  # 第i个相点到其最近距离点的距离
    Loc_DK = 1  # 第i个相点对应的最近距离点的下标

    for i in range(P + 1, M - 1):  # 限制短暂分离，从点P+1开始搜索
        d = 0
        for k in range(m):
            d = d + (Y[k][i] - Y[k][0]) * (Y[k][i] - Y[k][0])
        d = math.sqrt(d)
        if (d < DK) and (d > min_eps):
            DK = d
            Loc_DK = i
    #    以下计算各相点对应的李氏数保存到lmd()数组中
    #     i 为相点序号，从1到(M-1)，也是i-1点的演化点；Loc_DK为相点i-1对应最短距离的相点位置，DK为其对应的最短距离
    #     Loc_DK+1为Loc_DK的演化点，DK1为i点到Loc_DK+1点的距离，称为演化距离
    #     前i个log2（DK1/DK）的累计和用于求i点的lambda值
    sum_lmd = 0  # 存放前i个log2（DK1/DK）的累计和
    lmd = np.zeros(M - 1)
    for i in range(1, M - 1):  # 计算演化距离
        DK1 = 0
        for k in range(m):
            DK1 = DK1 + (Y[k][i] - Y[k][Loc_DK + 1]) * (Y[k][i] - Y[k][Loc_DK + 1])
        DK1 = math.sqrt(DK1)
        old_Loc_DK = Loc_DK  # 保存原最近位置相点
        old_DK = DK

        #    计算前i个log2（DK1/DK）的累计和以及保存i点的李氏指数
        if (DK1 != 0) and (DK != 0):
            sum_lmd = sum_lmd + np.log(DK1 / DK) / np.log(2)

        lmd[i - 1] = sum_lmd / [i]
        #  以下寻找i点的最短距离：要求距离在指定距离范围内尽量短，与DK1的角度最小
        point_num = 0  # 在指定距离范围内找到的候选相点的个数
        cos_sita = 0  # 夹角余弦的比较初值 ——要求一定是锐角
        zjfwcs = 0  # 增加范围次数
        while (point_num == 0):
            # 搜索相点
            for j in range(M - 1):
                if abs(j - i) <= (P - 1):  # 候选点距当前点太近，跳过！
                    continue
                    # 计算候选点与当前点的距离
                dnew = 0
                for k in range(m):
                    dnew = dnew + (Y[k][i] - Y[k][j]) * (Y[k][i] - Y[k][j])
                dnew = math.sqrt(dnew)
                if (dnew < min_eps) or (dnew > max_eps):  # 不在距离范围，跳过！
                    continue
                    # 计算夹角余弦及比较
                DOT = 0
                for k in range(m):
                    DOT = DOT + (Y[k][i] - Y[k][j]) * (Y[k][i] - Y[k][old_Loc_DK + 1])
                CTH = DOT / (dnew * DK1)
                # print(CTH)
                if CTH > 1.0:
                    CTH = 1.0
                if math.acos(CTH) > (3.14151926 / 4):  # 不是小于45度的角，跳过！
                    continue
                if CTH > cos_sita:  # 新夹角小于过去已找到的相点的夹角，保留
                    cos_sita = CTH
                    Loc_DK = j
                    DK = dnew
                point_num = point_num + 1

            if point_num <= min_point:
                max_eps = max_eps + dlt_eps
                zjfwcs = zjfwcs + 1
                if zjfwcs > MAX_CISHU:  # 超过最大放宽次数，改找最近的点
                    DK = 1.0e+100
                    for ii in range(M - 1):
                        if abs(i - ii) <= (P - 1):  # 候选点距当前点太近，跳过！
                            continue
                        d = 0
                        for k in range(m):
                            d = d + (Y[k][i] - Y[k][ii]) * (Y[k][i] - Y[k][ii])
                        d = math.sqrt(d)
                        if (d < DK) and (d > min_eps):
                            DK = d
                            Loc_DK = ii
                    break
                point_num = 0  # %&&扩大距离范围后重新搜索
                cos_sita = 0
    # 取平均得到最大李雅普诺夫指数
    # plt.plot(lmd)
    # plt.show()
    lambda_1 = sum(lmd) / len(lmd)
    # lambda_2 = max(lmd)
    # print("1")
    return lambda_1

In [3]:
def reconstitution(data, m, tau):
    """
    %该函数用来重构相空间
    % m为嵌入空间维数
    % tau为时间延迟
    % data为输入时间序列
    % N为时间序列长度
    % X为输出,是m*M维矩阵
    """
    N = len(data)
    M = N - (m - 1) * tau  # 相空间中点的个数
    X = np.zeros((m, M))
    for j in range(M):
        for i in range(m):
            X[i][j] = data[(i - 1) * tau + j]
    return X

In [4]:
def Max_lyapunov(data):
    m = 10
    tau = 2
    N = len(data)
    print("N", N)
    P = 10
    lambda_1 = lyapunov_wolf(data, N, m, tau, P)
    return lambda_1

In [11]:
import scipy

student_data_path = "cat/1"
data = scipy.io.loadmat(student_data_path)
print(data.keys())
temp = data["djc_eeg11"]

tt1 = time.time()
print(tt1)
t = Max_lyapunov(temp[4][1000:2000])
print(t)
tt2 = time.time()

t_sum = tt2-tt1
print("t_sum", t_sum)


dict_keys(['__header__', '__version__', '__globals__', 'djc_eeg1', 'djc_eeg2', 'djc_eeg3', 'djc_eeg4', 'djc_eeg5', 'djc_eeg6', 'djc_eeg7', 'djc_eeg8', 'djc_eeg9', 'djc_eeg10', 'djc_eeg11', 'djc_eeg12', 'djc_eeg13', 'djc_eeg14', 'djc_eeg15'])
1567413407.8080752
N 1000
0.06666568922402255
t_sum 12.591797590255737


In [12]:
def compute_DE(signal):
    variance = np.var(signal, ddof=1)
    return math.log(2*math.pi*math.e*variance)/2

In [13]:
tt1 = time.time()
print(tt1)
t = compute_DE(temp[4][1000:2000])
print(t)
tt2 = time.time()

t_sum = tt2-tt1
print("t_sum", t_sum)

1567428917.479011
4.4569761098470355
t_sum 0.0006401538848876953


In [14]:
def wgn(x, snr):  ## 功率谱密度？
    snr = 10**(snr/10.0)
    xpower = np.sum(x**2)/len(x)
    npower = xpower / snr
    return np.random.randn(len(x)) * np.sqrt(npower)

In [16]:
tt1 = time.time()
print(tt1)
t = wgn(temp[4][1000:2000],10)
print(t.shape)
tt2 = time.time()

t_sum = tt2-tt1
print("t_sum", t_sum)

1567429663.4954758
(1000,)
t_sum 0.0019202232360839844
